In [1]:
from pydrake.all import (SceneGraphCollisionChecker, 
                         Rgba, 
                         Sphere, 
                         RigidTransform, 
                         RotationMatrix,
                         VisibilityGraph)
from iris_environments.environments import plant_builder_2dof_triangle_with_hole
import ipywidgets as widgets
from functools import partial
from cspace_utils.plotting import plot_points
from cspace_utils.graphs import (greedy_clique_cover, greedy_geometric_clique_cover)
from cspace_utils.environment_helpers import get_sample_cfree_handle
import numpy as np
from cspace_utils.plotting import (plot_points, 
                                   plot_vpoly_2d_meshcat, 
                                   plot_edges_clique,
                                   plot_visibility_graph)
from cspace_utils.colors import generate_maximally_different_colors
plant, scene_graph, diagram, diagram_context, plant_context,meshcat = plant_builder_2dof_triangle_with_hole(True)
# meshcat.SetProperty('/drake/proximity','visible', True)
scene_graph_context = scene_graph.GetMyMutableContextFromRoot(
    diagram_context)
robot_instances = ['cspace_robot']
checker = SceneGraphCollisionChecker(model = diagram, 
                                     robot_model_instances = [plant.GetModelInstanceByName(r) for r in robot_instances], 
                                     edge_step_size = 0.1)

meshcat.Set2dRenderMode(RigidTransform(RotationMatrix.MakeYRotation(-np.pi/2), np.array([0,0,0.5])), xmin=-7, xmax=5, ymin=-5, ymax=7)
meshcat.SetProperty('/Lights/PointLightNegativeX', 'visible', False)
meshcat.SetProperty('/Lights/PointLightPositiveX', 'visible', False)
meshcat.SetProperty('/Lights/AmbientLight/<object>', 'intensity', 3.0)
# meshcat.SetProperty('/drake/proximity', 'visible', True)

INFO:drake:Meshcat listening for connections at http://localhost:7000
INFO:drake:Allocating contexts to support implicit context parallelism 20


http://localhost:7000


In [2]:
q = np.zeros(plant.num_positions()) 
sliders = []
for i in range(plant.num_positions()):
    q_low = plant.GetPositionLowerLimits()[i]*0.99
    q_high = plant.GetPositionUpperLimits()[i]*0.99
    sliders.append(widgets.FloatSlider(min=q_low, max=q_high, value=0, step=0.001, description=f"q{i}"))

col_col =  Rgba(0.8, 0.0, 0, 0.5)    
col_free =  Rgba(0.0, 0.8, 0.5, 0.5) 
def showres(qvis):
    plant.SetPositions(plant_context, qvis)
    diagram.ForcedPublish(diagram_context)
    query = plant.get_geometry_query_input_port().Eval(plant_context)
    col = query.HasCollisions()
    if col:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(1),
                                   col_col)
    else:
        meshcat.SetObject(f"/drake/visualizer/shunk",
                                   Sphere(1),
                                   col_free)
    meshcat.SetTransform(f"/drake/visualizer/shunk",
                                   RigidTransform(RotationMatrix(),
                                                  np.array([5,5,0])))
    return col

def handle_slider_change(change, idx):
    q[idx] = change['new']
    showres(q)
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)

FloatSlider(value=0.0, description='q0', max=5.9399999999999995, min=-5.9399999999999995, step=0.001)

FloatSlider(value=0.0, description='q1', max=5.9399999999999995, min=-5.9399999999999995, step=0.001)

In [3]:
N = 200

def col_free(q):
    return not checker.CheckConfigCollisionFree(q)

cfree_sampler = get_sample_cfree_handle(plant.GetPositionLowerLimits(), 
                                plant.GetPositionUpperLimits(),
                                col_free)
for i in range(20):
    meshcat.Delete(f'/drake/cl_{i}')
samps,_ = cfree_sampler(N, 1000,[])
adj = VisibilityGraph(checker, samps.T, parallelize = True)
# plot_points(meshcat, samps, 'nodes', color = Rgba(0.3,0,0,1), size = 0.1)

In [6]:

# clique_cover = greedy_clique_cover(adj, 
#                                    min_clique_size=1,
#                                    approach = 'mip')
clique_cover = greedy_geometric_clique_cover(adj, 
                                             samps.T, 
                                             min_gain_per_clique = 1,
                                             approach = 'greedy')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [5]:

from pydrake.all import VPolytope
for i in range(20):
    meshcat.Delete(f'/drake/cl_{i}')
nodes_for_plot = np.concatenate((samps, np.zeros((samps.shape[0],1))), axis = 1) 

colors = generate_maximally_different_colors(len(clique_cover))
colors =[Rgba(c[0], c[1],c[2],1) for c in colors]
plot_size_edge = 0.05
for idx, clique in enumerate(clique_cover):
    col = colors[idx]
    plot_points(meshcat, 
                samps[clique], 
                f'cl_{idx}',           
                color = col, 
                size = plot_size_edge*0.95)
    #plot_edges_clique(meshcat, clique, nodes_for_plot, f'cl_{idx}', col)
    if len(clique)>2:
        cliquevpoly = VPolytope(samps[clique].T).GetMinimalRepresentation()
        plot_vpoly_2d_meshcat(meshcat, 
                              cliquevpoly, 
                              f'cl_{idx}', 
                              col, 
                              size =plot_size_edge,
                              translation = np.array([0,0,0.1*idx]))
    elif len(clique)==2:
        plot_edges_clique(meshcat, 
                          clique, 
                          nodes_for_plot, 
                          f'cl_{idx}', 
                          col, 
                          size = plot_size_edge)
    else:
        plot_points(meshcat, 
                    samps[clique], 
                    f'cl_{idx}', 
                    color = col, 
                    size = plot_size_edge)
# plot_points(meshcat, nodes_for_plot, 'nodes', size=0.05, color = Rgba(0,0,0,1))
# plot_visibility_graph(meshcat, nodes_for_plot, adj, 'vg')

In [14]:
for s in samps[clique_cover[-1]]:
    print(s)

[-0.1189814   1.07457112]


In [6]:
plot_vpoly_2d_meshcat(meshcat,)

[Rgba(r=0.8, g=0.07999999999999999, b=0.07999999999999999, a=1.0),
 Rgba(r=0.07999999999999999, g=0.07999999999999999, b=0.8, a=1.0),
 Rgba(r=0.07999999999999999, g=0.8, b=0.8, a=1.0),
 Rgba(r=0.07999999999999999, g=0.8, b=0.07999999999999999, a=1.0),
 Rgba(r=0.8, g=0.07999999999999999, b=0.8, a=1.0),
 Rgba(r=0.8, g=0.8, b=0.07999999999999999, a=1.0)]

In [10]:
cliquevpoly.vertices()

array([[0.26098931],
       [0.32689015]])